In [ ]:
# !pip install git+https://github.com/ChatAGH/ChatAGH_RAG#
import json

import pandas as pd
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

from src.scraper import Scraper
from chat_agh.vector_store.mongodb import MongoDBVectorStore

DATA_PATH = "graph.json"

load_dotenv(
    dotenv_path="/Users/wnowogorski/PycharmProjects/ChatAGH/DataCollecting/.env"
)

df = pd.read_json(DATA_PATH)
df

/Users/wnowogorski/Library/Caches/pypoetry/virtualenvs/chatagh-data-collecting-fbzQknWL-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
python-dotenv could not parse statement starting at line 2


In [2]:
unique_urls = df["source"].unique()
len(unique_urls)

6382

In [3]:
clusters = df["cluster"].unique()
len(clusters)

7

In [6]:
def process_cluster(df, cluster_num):
    output_path = f"cluster_{cluster_num}/"

    df_subset = df[df["cluster"] == cluster_num]
    unique_urls = df_subset["source"].unique()
    unique_urls = ["https://" + url for url in unique_urls]
    scraper = Scraper(urls=unique_urls[:5], output_path=output_path)
    scraper.scrape()
    scraper.save_result_to_json("docs.json")

    with open(f"{output_path}/docs.json", "w") as file:
        docs_json = json.load(file)

    documents = [
        Document(page_content=doc["page_content"], metadata=doc["metadata"])
        for doc in docs_json
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
    chunks = splitter.split_documents(documents)

    vector_store = MongoDBVectorStore(collection_name=f"cluster_{cluster_num}")
    vector_store.indexing(chunks)

    failed_urls = [url.removeprefix("https://") for url in scraper.failed_urls]
    df = df[~(df["source"].isin(failed_urls) | df["target"].isin(failed_urls))]

    return df

In [5]:
for cluster in clusters:
    print("Processing cluster: ", cluster)
    df = process_cluster(df, cluster)

Processing cluster:  3
Processing url 1/5
Processing url 2/5
Processing url 3/5
Failed to process url: Error processing HTML content: No content found on the page.
Processing url 4/5
Processing url 5/5


UnsupportedOperation: not readable